In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import json
import requests
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import math
import statistics
import sys

In [91]:
draft_df = pd.read_excel("C:\\Users\\mcyee\\Desktop\\Untitled Folder\\2021_Fantasy_Football_Draft.xlsx")
draft_df.head()

,pick,player_name,player_position,player_team,salary,manager,keeper_code
0,1,Aaron Jones,RB,GB,27,Matt,BCDE
1,2,Jonathan Taylor,RB,IND,64,Matt,E
2,3,Odell Beckham Jr.,WR,CLE,5,Matt,E
3,4,Jerry Jeudy,WR,DEN,5,Matt,E
4,5,Davante Adams,WR,GB,60,Dai,ABCDE


In [17]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')

In [78]:
transaction_list = []
driver = webdriver.Chrome('C:\\Users\\mcyee\\Desktop\\Untitled Folder\\chromedriver.exe')

for n in np.arange(0, 20):
    try:
        p = str(n*25)
        driver.get("https://football.fantasysports.yahoo.com/f1/597209/transactions?transactionsfilter=all&count=" + p)
        time.sleep(2)
        transaction_html = driver.page_source
        transaction_soup = BeautifulSoup(transaction_html, "html.parser")
        transaction_table = transaction_soup.findAll("table", {"class": "Table Table-mid Tst-transaction-table"})[0].findAll("tbody")[0]
        for tr in transaction_table.findAll('tr'):
            td = tr.findAll('td')
            if str(td[0]).find("Added Player") != -1 and str(td[0]).find("Dropped Player") != -1:
                player_added = td[1].find_all("a", {'target': 'sports'})[0].text
                player_added_team = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[0]
                player_added_pos = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[1]
                player_dropped = td[1].find_all("a", {'target': 'sports'})[1].text
                player_dropped_team = td[1].find_all('span', {'class': 'F-position'})[1].text.split(' - ')[0]
                player_dropped_pos = td[1].find_all('span', {'class': 'F-position'})[1].text.split(' - ')[1]
                manager = td[2].find_all("a", {'class': 'Tst-team-name'})[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                transaction_details = td[1].find_all('h6', {'class': 'F-shade'})[0].text
                if transaction_details.find("$") != -1:
                    transaction_cost = (transaction_details.replace('$', '').replace(' ', '').replace('Waiver', ''))
                else:
                    transaction_cost = 0    
                transaction_dict = {
                    'transaction_type': 'drop-add',
                    'player_added': player_added,
                    'player_added_team': player_added_team,
                    'player_added_pos': player_added_pos,
                    'player_dropped': player_dropped,
                    'player_dropped_team': player_dropped_team,
                    'player_dropped_pos': player_dropped_pos,
                    'ff_team': manager,
                    'transaction_time': dt_transaction,
                    'transaction_cost': transaction_cost
                }
                transaction_list.append(transaction_dict)
            elif str(td[0]).find("Dropped Player") != -1:
                player_dropped = td[1].find_all("a", {'target': 'sports'})[0].text
                player_dropped_team = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[0]
                player_dropped_pos = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[1]
                manager = td[2].find_all("a", {'class': 'Tst-team-name'})[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                transaction_dict = {
                    'transaction_type': 'drop',
                    'player_added': 'Bench',
                    'player_added_team': 'N/A',
                    'player_added_pos': 'N/A',
                    'player_dropped': player_dropped,
                    'player_dropped_team': player_dropped_team,
                    'player_dropped_pos': player_dropped_pos,
                    'ff_team': manager,
                    'transaction_time': dt_transaction,
                    'transaction_cost': 0
                }
                transaction_list.append(transaction_dict)
            elif str(td[0]).find("Added Player") != -1:
                player_added = td[1].find_all("a", {'target': 'sports'})[0].text
                player_added_team = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[0]
                player_added_pos = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[1]
                manager = td[2].find_all("a", {'class': 'Tst-team-name'})[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                transaction_details = td[1].find_all('h6', {'class': 'F-shade'})[0].text
                if transaction_details.find("$") != -1:
                    transaction_cost = (transaction_details.replace('$', '').replace(' ', '').replace('Waiver', ''))
                else:
                    transaction_cost = 0 
                transaction_dict = {
                    'transaction_type': 'add',
                    'player_added': player_added,
                    'player_added_team': player_added_team,
                    'player_added_pos': player_added_pos,
                    'player_dropped': 'Bench',
                    'player_dropped_team': "N/A",
                    'player_dropped_pos': "N/A",
                    'ff_team': manager,
                    'transaction_time': dt_transaction,
                    'transaction_cost': transaction_cost
                }
                transaction_list.append(transaction_dict)

            elif str(td[1]).find("Traded to") != -1:
                manager = td[2].find_all("span", {'class': 'Grid-u'})[0].find_all("a")[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                adds = td[0].find_all('a', {'target': 'sports'})
                for a in range(len(adds)):
                    player_added = td[0].find_all("a", {'target': 'sports'})[a].text
                    player_added_team = td[0].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[0]
                    player_added_pos = td[0].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[1]
                    transaction_dict = {
                        'transaction_type': 'trade',
                        'player_added': player_added,
                        'player_added_team': player_added_team,
                        'player_added_pos': player_added_pos,
                        'player_dropped': 'N/A',
                        'player_dropped_team': "N/A",
                        'player_dropped_pos': "N/A",
                        'ff_team': manager,
                        'transaction_time': dt_transaction,
                        'transaction_cost': 0
                    }
                    transaction_list.append(transaction_dict)
            elif str(td[2]).find("Traded to") != -1:
                manager = td[3].find_all("span", {'class': 'Grid-u'})[0].find_all("a")[0].text
                dt_transaction = td[3].find_all("span", {'class': 'F-timestamp'})[0].text
                adds = td[1].find_all('a', {'target': 'sports'})
                for a in range(len(adds)):
                    player_added = td[1].find_all("a", {'target': 'sports'})[a].text
                    player_added_team = td[1].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[0]
                    player_added_pos = td[1].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[1]
                    transaction_dict = {
                        'transaction_type': 'trade',
                        'player_added': player_added,
                        'player_added_team': player_added_team,
                        'player_added_pos': player_added_pos,
                        'player_dropped': 'N/A',
                        'player_dropped_team': "N/A",
                        'player_dropped_pos': "N/A",
                        'ff_team': manager,
                        'transaction_time': dt_transaction,
                        'transaction_cost': 0
                    }
                    transaction_list.append(transaction_dict)
    except:
        pass

driver.quit()
transaction_df = pd.DataFrame(transaction_list)
transaction_df.head()

0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475


,ff_team,player_added,player_added_pos,player_added_team,player_dropped,player_dropped_pos,player_dropped_team,transaction_cost,transaction_time,transaction_type
0,folklore,Los Angeles,DEF,LAC,Philadelphia,DEF,Phi,0,"Nov 10, 11:02 am",drop-add
1,Odellta Variant,Le'Veon Bell,RB,Bal,Dan Arnold,TE,Jax,0,"Nov 10, 8:52 am",drop-add
2,Odellta Variant,Travis Etienne,RB,Jax,Boston Scott,RB,Phi,0,"Nov 10, 8:51 am",drop-add
3,Zeke and Ye Shall Find,Rhamondre Stevenson,RB,NE,Carlos Hyde,RB,Jax,0,"Nov 10, 6:43 am",drop-add
4,folklore,Philadelphia,DEF,Phi,San Francisco,DEF,SF,0,"Nov 10, 5:00 am",drop-add
5,Tiz the Law,Rashaad Penny,RB,Sea,Dallas,DEF,Dal,0,"Nov 10, 4:31 am",drop-add
6,Mediocre Team,Eno Benjamin,RB,Ari,Cole Kmet,TE,Chi,0,"Nov 10, 4:20 am",drop-add
7,Mediocre Team,Deonte Harris,WR,NO,James Washington,WR,Pit,0,"Nov 10, 4:19 am",drop-add
8,Mediocre Team,Younghoe Koo,K,Atl,Brandon McManus,K,Den,0,"Nov 10, 4:18 am",drop-add
9,Mediocre Team,Denver,DEF,Den,Chicago,DEF,Chi,0,"Nov 10, 4:15 am",drop-add


In [82]:
manager_list = []
for m in list(set(transaction_df['ff_team'])):
    manager = input("Which player is the manager for " + m + "?")
    manager_dict = {
        'ff_team': m,
        'manager': manager
    }
    manager_list.append(manager_dict)
manager_df = pd.DataFrame(manager_list)
manager_df
    

Which player is the manager for Me So Fourney?Joel
Which player is the manager for Raider Bae?Jake
Which player is the manager for Nags?Ryan
Which player is the manager for Odellta Variant?Matt
Which player is the manager for Boswell that Ends Well?Sean
Which player is the manager for Chi ShingT's Team?Chi Shing
Which player is the manager for Pop Drop and Lockett?Evan
Which player is the manager for Tiz the Law?Dai
Which player is the manager for G?Jiwei
Which player is the manager for Zeke and Ye Shall Find?Doug
Which player is the manager for Mediocre Team?Rajiv
Which player is the manager for folklore?Ron


,ff_team,manager
0,Me So Fourney,Joel
1,Raider Bae,Jake
2,Nags,Ryan
3,Odellta Variant,Matt
4,Boswell that Ends Well,Sean
5,Chi ShingT's Team,Chi Shing
6,Pop Drop and Lockett,Evan
7,Tiz the Law,Dai
8,G,Jiwei
9,Zeke and Ye Shall Find,Doug


In [84]:
transaction_df = pd.DataFrame(transaction_list)
transaction_df['transaction_time'] = "2020 " + transaction_df['transaction_time']
transaction_df['transaction_time'] = pd.to_datetime(transaction_df['transaction_time'], format="%Y %b %d, %I:%M %p")
transaction_df = transaction_df.sort_values(by='transaction_time').reset_index(drop=True)
transaction_df = transaction_df.merge(manager_df, on='ff_team', how='left')
transaction_df.to_csv("C:\\Users\\mcyee\\Desktop\\Untitled Folder\\2021_Fantasy_Football_Transactions.csv")
transaction_df.head()

,ff_team,player_added,player_added_pos,player_added_team,player_dropped,player_dropped_pos,player_dropped_team,transaction_cost,transaction_time,transaction_type,manager
0,Pop Drop and Lockett,Ty'Son Williams,RB,Bal,Los Angeles,DEF,LAC,7,2020-08-30 00:09:00,drop-add,Evan
1,Me So Fourney,Jermar Jefferson,RB,Det,Kirk Cousins,QB,Min,5,2020-08-30 00:09:00,drop-add,Joel
2,Odellta Variant,Nelson Agholor,WR,NE,Todd Gurley II,RB,Atl,0,2020-08-30 00:09:00,drop-add,Matt
3,Boswell that Ends Well,Sterling Shepard,WR,NYG,Russell Gage,WR,Atl,0,2020-08-30 00:10:00,drop-add,Sean
4,Boswell that Ends Well,Tony Jones Jr.,RB,NO,Latavius Murray,RB,Bal,0,2020-08-30 00:13:00,drop-add,Sean


In [81]:
transaction_df.columns

Index(['ff_team', 'player_added', 'player_added_pos', 'player_added_team',
       'player_dropped', 'player_dropped_pos', 'player_dropped_team',
       'transaction_cost', 'transaction_time', 'transaction_type'],
      dtype='object')

In [123]:
draft_df = pd.read_excel("C:\\Users\\mcyee\\Desktop\\Untitled Folder\\2021_Fantasy_Football_Draft.xlsx")
draft_df.head()

for i, r in transaction_df.iterrows():
#     print(i, r['transaction_type'], r['player_added'], r['player_added_team'], r['player_dropped'], r['player_dropped_team'])
    if r['transaction_type'] == 'drop-add':
        draft_row_n = draft_df[(draft_df['player_name'] == r['player_dropped']) & (draft_df['player_team'] == r['player_dropped_team'])]
        draft_df.at[draft_row_n.index[0], 'player_name'] = r['player_added']
        draft_df.at[draft_row_n.index[0], 'player_team'] = r['player_added_team']
        draft_df.at[draft_row_n.index[0], 'player_position'] = r['player_added_pos']
        draft_df.at[draft_row_n.index[0], 'salary'] = r['transaction_cost']
        draft_df = draft_df.reset_index(drop=True)
    elif r['transaction_type'] == 'drop':
        draft_row_n = draft_df[(draft_df['player_name'] == r['player_dropped']) & (draft_df['player_team'] == r['player_dropped_team'])]
        draft_df = draft_df.drop([draft_row_n.index[0]]).reset_index(drop=True)
    elif r['transaction_type'] == 'add':
        draft_df = draft_df.append({'pick': 0, 
                         'player_name': r['player_added'], 
                         'player_position': r['player_added_pos'], 
                         'player_team': r['player_added_team'],
                         'salary': r['transaction_cost'],
                         'manager': r['manager'],
                         'keeper_code': ''
                        }, ignore_index=True)
        draft_df = draft_df.reset_index(drop=True)
    elif r['transaction_type'] == 'trade':
        draft_row_n = draft_df[(draft_df['player_name'] == r['player_added']) & (draft_df['player_team'] == r['player_added_team'])]
        draft_df.at[draft_row_n.index[0], 'manager'] = r['manager']
        draft_df.reset_index(drop=True)


In [124]:
draft_df.to_csv('C:\\Users\\mcyee\\Desktop\\Untitled Folder\\2021_Current_Roster.csv')